In [17]:
# Problem 1
from pulp import *
import numpy as np
import pandas as pd

# Constants
month = list(range(1,7)) # month
initial_balance = 0.4  # Starting cash balance (in $100,000)
min_balance = 0.25  # Minimum cash balance required (in $100,000)
receivables = [1.5, 1.0, 1.4, 2.3, 2.0, 1.0]  # Monthly accounts receivables (in $100,000)
monthly_rec = dict(zip(month,receivables))
payments = [1.8, 1.6, 2.2, 1.2, 0.8, 1.2]  # Monthly payments due (in $100,000)
monthly_pay = dict(zip(month,payments))
delay_discount = 1/(1-0.02)# Delay payment discount rate
interest_carry = 0.005  # Interest on cash carried over
interest_borrow = 0.015  # Interest on borrowed amount
borrow_limit = 0.75  # Maximum proportion of receivables that can be borrowed
interest_loan = 0.01  # Interest on short-term loan

# Decision Variables
# Amount borrowed each month
borrow_amount = LpVariable.dicts("Borrow", month, lowBound=0, cat='Continuous')
# Amount of payment delayed each month
payment_delay = LpVariable.dicts("Delayed_Payment", month, lowBound=0, cat='Continuous')
# Short-term loan taken at the beginning
shortterm_loan = LpVariable("Loan", lowBound=0, cat='Continuous')

#Dependent Variables
payment = LpVariable.dicts("Payment", month, lowBound=0, cat='Continuous')
monthly_balance = LpVariable.dicts("Month_Balance", month, lowBound=min_balance, cat='Continuous')

# Set the problem
prob = LpProblem("Assignment_4.1",LpMinimize)

# Objective function: Minimize the total cost
total_delay_discount = lpSum([payment_delay[i] + payment[i] - monthly_pay[i] for i in month])
borrow_interest = lpSum([borrow_amount[i] * interest_borrow for i in month[:-1]])
interest_earned = (initial_balance + lpSum([monthly_balance[i] for i in month[:-1]])) * interest_carry
prob += total_delay_discount + borrow_interest + shortterm_loan*((1+interest_loan)**6) - interest_earned

# Constraints
#Month 1
prob += initial_balance * (1 + interest_carry) + monthly_rec[1] + borrow_amount[1] + shortterm_loan == payment[1] + interest_loan * shortterm_loan + monthly_balance[1]
#Month 2-5
for i in range(2, 6):
    prob += monthly_balance[i - 1] * (1 + interest_carry) + monthly_rec[i] + borrow_amount[i] == payment_delay[i - 1] + payment[i] + (1 + interest_borrow) * borrow_amount[i - 1] + interest_loan * shortterm_loan + monthly_balance[i]
#Month 6
prob += monthly_balance[5] * (1 + interest_carry) + monthly_rec[6] + borrow_amount[6] == payment_delay[5] + payment[6] + (1 + interest_borrow) * borrow_amount[5] + (1 + interest_loan) * shortterm_loan + monthly_balance[6]
prob += borrow_amount[6] == 0
prob += payment_delay[6] == 0
prob += payment[6] == monthly_pay[6]
#Decision Var constraint
for i in month:
    prob += borrow_amount[i] <= borrow_limit * monthly_rec[i]
    prob += payment[i] == monthly_pay[i]
    prob += payment_delay[i] == (monthly_pay[i] - payment[i]) * delay_discount

# Solve
prob.writeLP("BalanceManagement.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

#Check the variables and constraints
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Objective=", value(prob.objective))

# Results
output = []
for i in month:
    var_output = [borrow_amount[i].varValue, payment[i].varValue, payment_delay[i].varValue, monthly_balance[i].varValue]
    output.append(var_output)
col_names = ['Borrow', 'Payment', 'Payment Delayed', 'Monthly Balance']
print("One-time six-month loan =", shortterm_loan.varValue)
print("Net Financial Cost =", value(prob.objective))
output_df = pd.DataFrame(output, index=[1, 2, 3, 4, 5, 6], columns=col_names)
output_df

Status: Optimal
Borrow_1 = 0.0
Borrow_2 = 0.242486
Borrow_3 = 1.05
Borrow_4 = 0.0
Borrow_5 = 0.0
Borrow_6 = 0.0
Delayed_Payment_1 = 0.0
Delayed_Payment_2 = 0.0
Delayed_Payment_3 = 0.0
Delayed_Payment_4 = 0.0
Delayed_Payment_5 = 0.0
Delayed_Payment_6 = 0.0
Loan = 0.512721
Month_Balance_1 = 0.609594
Month_Balance_2 = 0.25
Month_Balance_3 = 0.25
Month_Balance_4 = 0.280373
Month_Balance_5 = 1.47665
Month_Balance_6 = 0.766183
Payment_1 = 1.8
Payment_2 = 1.6
Payment_3 = 2.2
Payment_4 = 1.2
Payment_5 = 0.8
Payment_6 = 1.2
Objective= 0.5473178781362948
One-time six-month loan = 0.512721
Net Financial Cost = 0.5473178781362948


,Borrow,Payment,Payment Delayed,Monthly Balance
1,0.000000,1.8,0.0,0.609594
2,0.242486,1.6,0.0,0.250000
3,1.050000,2.2,0.0,0.250000
4,0.000000,1.2,0.0,0.280373
5,0.000000,0.8,0.0,1.476650
6,0.000000,1.2,0.0,0.766183
